### Emotions Extraction

In [7]:
# install the transformers library if nessasery.
# !pip install transformers
# !pip install xformers

In [2]:
# import required packages
import pandas as pd
from transformers import pipeline

In [3]:
# specify ther filename
file_name = "/data/notebook_files/Sample_Dataset.csv"  # note: you can right-click on your file and copy-paste the path to it here
text_column = "Description"  # select the column in the csv that contains the text to be classified
dataset = pd.read_csv(file_name)
dataset['index'] = dataset.index

In [5]:
dataset.sample(5)

In [6]:
pred_texts = dataset[text_column].dropna().astype('str').tolist() # Extract only the texts to list for the model.
pred_texts = list(map(lambda x: x.replace("\r",".").replace("\n",""), pred_texts)) # Removes Raw strings.

### Classify texts with model

In [ ]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
predictions = classifier(pred_texts)

In [13]:
# Process the data to create a list of dictionaries with the label as key and score as value
processed_data = [{item['label']: item['score'] for item in inner_list} for inner_list in predictions]

# Convert the processed data into a DataFrame
df = pd.DataFrame(processed_data)

df['score'] = df.max(axis=1)
df['label'] = df.apply(lambda row: row.idxmax(), axis=1)
df['index'] = dataset.index
df = df[['index','label','score', 'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']]

final_dataset = pd.merge(dataset, df, on='index',  how='left')
final_dataset.drop(columns=['index'], inplace=True)

In [14]:
final_dataset.label.value_counts()

In [15]:
final_dataset.sample(5)

### Export results

In [92]:
# save results to csv
FILENAME = "Fully_Sample_Dataset.csv"  # name your output file
final_dataset.to_csv(FILENAME)